In [34]:
import pandas as pd
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm

In [11]:
df = pd.read_csv('Comments.csv')
df_ch = pd.read_csv('Channels.csv')
df_filter = pd.read_csv('filter_words.csv')
morph_analyzer = MorphAnalyzer()

C:\Users\V\AppData\Local\Temp\ipykernel_19236\1496209372.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Comments.csv')


In [12]:
level1 = df_filter[df_filter['level'] == 1]['words'].apply(lambda s: morph_analyzer.parse(s)[0].normal_form).values.tolist()
level2 = df_filter[df_filter['level'] == 2]['words'].apply(lambda s: morph_analyzer.parse(s)[0].normal_form).values.tolist()

In [14]:
import re

def _stem_text(text, morph_analyzer):
    if type(text) != type('привет Андрей'):
        return ''
    text = text.lower()
    text = re.sub(r'@[0-9a-zа-яё\-. ]*', '', text)
    text = re.sub(r'[^a-zа-яё ]+', ' ', text)
    text = text.replace("  ", " ")
    return text

In [16]:
%%time

df_ch['title_pr'] = df_ch['title'].apply(lambda s: [morph_analyzer.parse(i)[0].normal_form for i in _stem_text(s, 1).split(' ')])
df_ch['description_pr'] = df_ch['description'].apply(lambda s: [morph_analyzer.parse(i)[0].normal_form for i in _stem_text(s, 1).split(' ')])

Wall time: 1min 3s


In [27]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [32]:
%%time


df_ch['tags_pr'] = df_ch['tags'].apply(lambda s: [morph_analyzer.parse(i)[0].normal_form for i in flatten([_stem_text(i, 1).split(' ') for i in s.split('\'')])])

Wall time: 59.6 s


In [35]:
lst = df['text'].values.tolist()
res = []

for j in tqdm(lst):
    res +=  [[morph_analyzer.parse(i)[0].normal_form for i in _stem_text(j, 1).split(' ')]]
    
df['text_pr'] = res

  0%|          | 0/1646130 [00:00<?, ?it/s]

In [37]:
df.to_csv('debug/comments.csv')
df_ch.to_csv('debug/channels.csv')

In [39]:
lps = df['text_pr'].values.tolist()

lv1_ = []
lv2_ = []

for i in tqdm(lps):
    lv1_ += [len(list(set(level1) & set(i)))]
    lv2_ += [len(list(set(level2) & set(i)))]
    
df['level1'] = lv1_
df['level2'] = lv2_

  0%|          | 0/1646130 [00:00<?, ?it/s]

In [115]:
#lps1 = df_ch['tags_pr'].values.tolist()
lps1 = [' _ '] * len(df_ch['tags_pr'].values.tolist())
lps2 = df_ch['description_pr'].values.tolist()
lps3 = df_ch['title_pr'].values.tolist()

lv1_ = []
lv2_ = []

for i in range(len(lps1)):
    lv1_ += [len(list(set(level1) & set(lps1[i]))) + len(list(set(level1) & set(lps2[i]))) + len(list(set(level1) & set(lps3[i])))]
    lv2_ += [len(list(set(level2) & set(lps1[i]))) + len(list(set(level2) & set(lps2[i]))) + len(list(set(level2) & set(lps3[i])))]

df_ch['level1'] = lv1_
df_ch['level2'] = lv2_

In [123]:
videos_1 = df_ch[(df_ch['level1'] + df_ch['level2'] >= 2)][['videoId', 'publicationDate']]

In [122]:
videos_1l = set(videos_1['videoId'].values.tolist())

In [109]:
spr = {'comment' : [], 'sentiment' : [], 'date' : [], 'id' : []}

for _, i in tqdm(df_.iterrows()):
    if (i['videoId'] in videos_1l) and (i['level1'] + i['level2'] > 0):
        spr['comment'] += [i['text']]
        spr['sentiment'] += [i['sentiment']]
        spr['date'] += [videos_1[videos_1['videoId'] == i['videoId']]['publicationDate'].values[0]]
        spr['id'] += [i['cid']]

0it [00:00, ?it/s]

In [110]:
df_YT = pd.DataFrame(data = spr)

In [126]:
df_YT['date'] = df_YT['date'].apply(lambda s: s[:10])

In [127]:
dates_ = pd.read_csv('date.csv')
dates_ = dates_['date'].values.tolist()
dates_ = dates_[::-1]
dates_ = ['-'.join(i.split('.')[::-1]) for i in dates_]

In [128]:
def f_ss(s):
    for i in range(len(dates_)):
        if s >= dates_[i]:
            return dates_[i]
        
    return '0000-00-00'

In [129]:
df_YT['date_processed'] = df_YT['date'].apply(lambda s: f_ss(s))

In [130]:
df_YT['positive'] = df_YT['sentiment'].apply(lambda s: 1 if s == 1 else 0)
df_YT['negative'] = df_YT['sentiment'].apply(lambda s: 1 if s == -1 else 0)
df_YT['neutral'] = df_YT['sentiment'].apply(lambda s: 1 if s == 0 else 0)

In [131]:
df_resulting = df_YT[['date_processed', 'positive', 'negative', 'neutral']].groupby(by=['date_processed']).sum().reset_index()

In [132]:
df_resulting.to_csv('YT_HARD_FILTER.csv')